# Tutorial: Build a regression model with Open Datasets

In this tutorial, you leverage the convenience of Azure Open Datasets to create a regression model to predict NYC taxi fare prices. Easily download publicly available taxi, holiday and weather data to create a dataset that can train a regression model using sklearn.

In [1]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.options.mode.chained_assignment = None

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


## Download Data
Begin by downloading the NYC Taxi dataset from Azure Open Datasets. In non-Spark environments, Open Datasets only allows one month of data at a time with certain classes to avoid MemoryError with large datasets. To download 1 year of taxi data, we will fetch 2000 random samples from each month.

Note: Open Datasets has mirroring classes for working in Spark where data size and memory are not a concern.

In [2]:
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

green_taxi_df = pd.concat([NycTlcGreen(start + relativedelta(months=x), end + relativedelta(months=x)) \
        .to_pandas_dataframe().sample(2000) for x in range(12)])
green_taxi_df

[Info] read from /tmp/tmpm9erjg7h/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2016/puMonth=1/part-00119-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2689-1.c000.snappy.parquet
[Info] read from /tmp/tmp9svrtdy0/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2016/puMonth=2/part-00060-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2630-2.c000.snappy.parquet
[Info] read from /tmp/tmp2h8fjxvw/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2016/puMonth=3/part-00196-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2766-1.c000.snappy.parquet
[Info] read from /tmp/tmpfvo7iz0i/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2016/puMonth=4/part-00121-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2691-1.c000.snappy.parquet
[Info] read from /tmp/tmpjlfw4v7s/https%3A/%2Fazureopendatastorage.azurefd.net/nyctlc/green/puYear=2016/puMonth=5/part-00044-tid-4753095944193949832

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
1312085,2,2016-01-03 11:10:13,2016-01-03 11:14:13,1,0.83,None,None,-73.939774,40.679844,-73.930649,...,2,5.0,0.0,0.5,0.3,0.00,0.0,NaN,5.80,1.0
109916,2,2016-01-19 08:11:09,2016-01-19 08:16:29,1,0.85,None,None,-73.925629,40.761787,-73.937866,...,2,5.5,0.0,0.5,0.3,0.00,0.0,NaN,6.30,1.0
25029,2,2016-01-02 11:47:40,2016-01-02 11:52:29,1,0.81,None,None,-73.973312,40.689678,-73.984985,...,1,5.0,0.0,0.5,0.3,1.16,0.0,NaN,6.96,1.0
629848,2,2016-01-17 18:31:30,2016-01-17 18:42:32,1,2.21,None,None,-73.928474,40.687298,-73.940605,...,2,9.5,0.0,0.5,0.3,0.00,0.0,NaN,10.30,1.0
139651,2,2016-01-23 00:00:17,2016-01-23 00:05:10,1,0.60,None,None,-73.953415,40.706947,-73.948738,...,2,5.0,0.5,0.5,0.3,0.00,0.0,NaN,6.30,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44592,1,2016-12-05 08:14:48,2016-12-05 08:39:17,1,3.70,49,71,NaN,NaN,NaN,...,2,17.5,0.0,0.5,0.3,0.00,0.0,NaN,18.30,1.0
731527,2,2016-12-24 00:07:40,2016-12-24 00:10:19,1,0.47,255,255,NaN,NaN,NaN,...,1,4.0,0.5,0.5,0.3,1.06,0.0,NaN,6.36,1.0
501002,1,2016-12-18 05:47:22,2016-12-18 06:10:34,1,8.40,116,79,NaN,NaN,NaN,...,1,27.0,0.5,0.5,0.3,5.65,0.0,NaN,33.95,1.0
700564,2,2016-12-23 12:49:47,2016-12-23 13:00:52,1,2.63,166,236,NaN,NaN,NaN,...,1,10.5,0.0,0.5,0.3,2.00,0.0,NaN,13.30,1.0


Now that the initial data is loaded, define a function to create various time-based features from the pickup datetime field. This will create new fields for the month number, day of month, day of week, and hour of day. From those, we calculate the sin and cosine transformations to capture the cyclical nature of the variable which will allow the model to factor in time-based seasonality. This function also adds a static feature for the country code to join the holiday data. Use the apply() function on the dataframe to interatively apply this function to each row in the dataframe.

In [3]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour
    country_code = "US"
    hr_sin = np.sin(hour_of_day*(2.*np.pi/24))
    hr_cos = np.cos(hour_of_day*(2.*np.pi/24))
    dy_sin = np.sin(day_of_week*(2.*np.pi/7))
    dy_cos = np.cos(day_of_week*(2.*np.pi/7))
    
    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day, country_code, hr_sin, hr_cos, dy_sin, dy_cos))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day", "country_code", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,tripType,month_num,day_of_month,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos
1312085,2,2016-01-03 11:10:13,2016-01-03 11:14:13,1,0.83,None,None,-73.939774,40.679844,-73.930649,...,1.0,1,3,6,11,US,2.588190e-01,-9.659258e-01,-0.781831,0.623490
109916,2,2016-01-19 08:11:09,2016-01-19 08:16:29,1,0.85,None,None,-73.925629,40.761787,-73.937866,...,1.0,1,19,1,8,US,8.660254e-01,-5.000000e-01,0.781831,0.623490
25029,2,2016-01-02 11:47:40,2016-01-02 11:52:29,1,0.81,None,None,-73.973312,40.689678,-73.984985,...,1.0,1,2,5,11,US,2.588190e-01,-9.659258e-01,-0.974928,-0.222521
629848,2,2016-01-17 18:31:30,2016-01-17 18:42:32,1,2.21,None,None,-73.928474,40.687298,-73.940605,...,1.0,1,17,6,18,US,-1.000000e+00,-1.836970e-16,-0.781831,0.623490
139651,2,2016-01-23 00:00:17,2016-01-23 00:05:10,1,0.60,None,None,-73.953415,40.706947,-73.948738,...,1.0,1,23,5,0,US,0.000000e+00,1.000000e+00,-0.974928,-0.222521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44592,1,2016-12-05 08:14:48,2016-12-05 08:39:17,1,3.70,49,71,NaN,NaN,NaN,...,1.0,12,5,0,8,US,8.660254e-01,-5.000000e-01,0.000000,1.000000
731527,2,2016-12-24 00:07:40,2016-12-24 00:10:19,1,0.47,255,255,NaN,NaN,NaN,...,1.0,12,24,5,0,US,0.000000e+00,1.000000e+00,-0.974928,-0.222521
501002,1,2016-12-18 05:47:22,2016-12-18 06:10:34,1,8.40,116,79,NaN,NaN,NaN,...,1.0,12,18,6,5,US,9.659258e-01,2.588190e-01,-0.781831,0.623490
700564,2,2016-12-23 12:49:47,2016-12-23 13:00:52,1,2.63,166,236,NaN,NaN,NaN,...,1.0,12,23,4,12,US,1.224647e-16,-1.000000e+00,-0.433884,-0.900969


Remove some of the columns that you won't need for modeling or additional feature building. Rename the time field for pickup time, and additionally convert the time to midnight using `pandas.Series.dt.normalize`. This is done to all time features so that the datetime column can be later used as a key when joining datasets together at a daily level of granularity.

In [4]:
columns_to_remove = ["lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"]

green_taxi_df.drop(columns_to_remove, axis=1, inplace=True)

green_taxi_df["datetime"] = green_taxi_df["lpepPickupDatetime"].dt.normalize()
green_taxi_df.head(5)

,vendorID,lpepPickupDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos,datetime
1312085,2,2016-01-03 11:10:13,1,0.83,-73.939774,40.679844,-73.930649,40.674252,5.80,1,3,6,11,US,0.258819,-9.659258e-01,-0.781831,0.623490,2016-01-03
109916,2,2016-01-19 08:11:09,1,0.85,-73.925629,40.761787,-73.937866,40.766113,6.30,1,19,1,8,US,0.866025,-5.000000e-01,0.781831,0.623490,2016-01-19
25029,2,2016-01-02 11:47:40,1,0.81,-73.973312,40.689678,-73.984985,40.688690,6.96,1,2,5,11,US,0.258819,-9.659258e-01,-0.974928,-0.222521,2016-01-02
629848,2,2016-01-17 18:31:30,1,2.21,-73.928474,40.687298,-73.940605,40.674679,10.30,1,17,6,18,US,-1.000000,-1.836970e-16,-0.781831,0.623490,2016-01-17
139651,2,2016-01-23 00:00:17,1,0.60,-73.953415,40.706947,-73.948738,40.711098,6.30,1,23,5,0,US,0.000000,1.000000e+00,-0.974928,-0.222521,2016-01-23


### Enrich with Holiday Data

Now that the taxi data is downloaded and roughly prepared, add in holiday data as additional features. Holiday-specific features will assist model accuracy, as major holidays are times where taxi demand increases dramatically and supply becomes limited. The holiday dataset is relatively small, so fetch the full set by using the `PublicHolidays` class constructor with no parameters for filtering. Preview the data to check the format.

In [5]:
from azureml.opendatasets import PublicHolidays

# call default constructor to download full dataset
holidays_df = PublicHolidays().to_pandas_dataframe()
holidays_df.head(5)

[Info] read from /tmp/tmpiw_16gzx/https%3A/%2Fazureopendatastorage.azurefd.net/holidaydatacontainer/Processed/part-00000-tid-8468414522853579044-35925ba8-a227-4b80-9c89-17065e7bf1db-649-c000.snappy.parquet


,countryOrRegion,holidayName,normalizeHolidayName,isPaidTimeOff,countryRegionCode,date
19375,Argentina,Año Nuevo [New Year's Day],Año Nuevo [New Year's Day],None,AR,2008-01-01
19376,Australia,New Year's Day,New Year's Day,None,AU,2008-01-01
19377,Austria,Neujahr,Neujahr,None,AT,2008-01-01
19378,Belarus,Новый год,Новый год,None,BY,2008-01-01
19379,Belgium,Nieuwjaarsdag,Nieuwjaarsdag,None,BE,2008-01-01


Rename the `countryRegionCode` and `date` columns to match the respective field names from the taxi data, and also normalize the time so it can be used as a key. Next, join the holiday data with the taxi data by performing a left-join using the Pandas `merge()` function. This will preserve all records from `green_taxi_df`, but add in holiday data where it exists for the corresponding `datetime` and `country_code`, which in this case is always `\"US\"`. Preview the data to verify that they were merged correctly.

In [6]:
holidays_df = holidays_df.rename(columns={"countryRegionCode": "country_code"})
holidays_df["datetime"] = holidays_df["date"].dt.normalize()

holidays_df.drop(["countryOrRegion", "holidayName", "date"], axis=1, inplace=True)

taxi_holidays_df = pd.merge(green_taxi_df, holidays_df, how="left", on=["datetime", "country_code"])
taxi_holidays_df[taxi_holidays_df["normalizeHolidayName"].notnull()]

,vendorID,lpepPickupDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,...,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos,datetime,normalizeHolidayName,isPaidTimeOff
10,2,2016-01-01 06:10:47,1,1.01,-73.937195,40.679676,-73.922226,40.680149,5.80,1,...,4,6,US,1.000000,6.123234e-17,-0.433884,-0.900969,2016-01-01,New Year's Day,True
12,2,2016-01-01 20:35:55,1,3.81,-73.881638,40.767544,-73.917046,40.769688,16.80,1,...,4,20,US,-0.866025,5.000000e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
20,2,2016-01-01 14:29:02,1,1.60,-73.950226,40.678459,-73.958611,40.698792,14.80,1,...,4,14,US,-0.500000,-8.660254e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
30,2,2016-01-01 22:38:33,1,0.60,-73.890671,40.746601,-73.896980,40.745064,5.30,1,...,4,22,US,-0.500000,8.660254e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
73,2,2016-01-18 14:45:12,1,1.02,-73.945190,40.792698,-73.935822,40.796143,6.30,1,...,0,14,US,-0.500000,-8.660254e-01,0.000000,1.000000,2016-01-18,Martin Luther King Jr. Day,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23721,2,2016-12-26 19:17:20,1,1.00,NaN,NaN,NaN,NaN,7.30,12,...,0,19,US,-0.965926,2.588190e-01,0.000000,1.000000,2016-12-26,Christmas Day,True
23843,2,2016-12-25 22:11:04,1,0.65,NaN,NaN,NaN,NaN,5.30,12,...,6,22,US,-0.500000,8.660254e-01,-0.781831,0.623490,2016-12-25,Christmas Day,True
23852,2,2016-12-26 23:48:40,1,2.84,NaN,NaN,NaN,NaN,12.30,12,...,0,23,US,-0.258819,9.659258e-01,0.000000,1.000000,2016-12-26,Christmas Day,True
23858,2,2016-12-25 02:19:58,3,6.83,NaN,NaN,NaN,NaN,27.30,12,...,6,2,US,0.500000,8.660254e-01,-0.781831,0.623490,2016-12-25,Christmas Day,True


### Enrich with weather data

Now NOAA surface weather data can be appended to the taxi and holiday data. Use a similar approach to fetch the weather data by downloading one month at a time iteratively. Additionally, specify the `cols` parameter with an array of strings to filter the columns to download. This is a very large dataset containing weather surface data from all over the world, so before appending each month, filter the lat/long fields to near NYC using the `query()` function on the dataframe. This will ensure the `weather_df` doesn't get too large.

In [7]:
from azureml.opendatasets import NoaaIsdWeather
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

weather_df = pd.concat([NoaaIsdWeather(cols=["temperature", "precipTime", "precipDepth"], start_date=start + relativedelta(months=x), end_date=end + relativedelta(months=x))\
        .to_pandas_dataframe().query("latitude>=40.53 and latitude<=40.88 and longitude>=-74.09 and longitude<=-73.72 and temperature==temperature") for x in range(12)])

[Info] read from /tmp/tmpcav0ogcg/https%3A/%2Fazureopendatastorage.azurefd.net/isdweatherdatacontainer/ISDWeather/year=2016/month=1/part-00000-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2623-1.c000.snappy.parquet
[Info] read from /tmp/tmpcav0ogcg/https%3A/%2Fazureopendatastorage.azurefd.net/isdweatherdatacontainer/ISDWeather/year=2016/month=1/part-00004-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2629-1.c000.snappy.parquet
[Info] read from /tmp/tmpcav0ogcg/https%3A/%2Fazureopendatastorage.azurefd.net/isdweatherdatacontainer/ISDWeather/year=2016/month=1/part-00001-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2628-1.c000.snappy.parquet
[Info] read from /tmp/tmpcav0ogcg/https%3A/%2Fazureopendatastorage.azurefd.net/isdweatherdatacontainer/ISDWeather/year=2016/month=1/part-00005-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2627-1.c000.snappy.parquet
[Info] read from /tmp/tmpcav0ogcg/https%3A/%2Fazureopendatastorage.azurefd.net/i

In [8]:
weather_df

,temperature,precipTime,latitude,longitude,datetime,wban,precipDepth,usaf
204647,2.8,NaN,40.783,-73.867,2016-01-02 03:00:00,14732,NaN,725030
204670,-4.4,1.0,40.779,-73.880,2016-01-22 13:51:00,14732,0.0,725030
204694,5.0,1.0,40.779,-73.880,2016-01-08 02:51:00,14732,0.0,725030
204701,-1.1,1.0,40.779,-73.880,2016-01-04 15:51:00,14732,0.0,725030
204715,4.4,1.0,40.779,-73.880,2016-01-01 21:51:00,14732,0.0,725030
...,...,...,...,...,...,...,...,...
1248471,4.4,1.0,40.789,-73.967,2016-12-23 13:51:00,94728,0.0,725053
1248555,5.0,1.0,40.789,-73.967,2016-12-12 13:51:00,94728,0.0,725053
1248580,3.9,NaN,40.789,-73.967,2016-12-18 07:01:00,94728,NaN,725053
1248597,7.8,1.0,40.789,-73.967,2016-12-25 00:51:00,94728,0.0,725053


Again call `pandas.Series.dt.normalize` on the `datetime` field in the weather data so it matches the time key in `taxi_holidays_df`.


Next group the weather data to have daily aggregated weather values. Define a dict `aggregations` to define how to aggregate each field at a daily level. For`temperature` take the mean and for `precipTime` and `precipDepth` take the daily maximum. Use the `groupby()` function along with the aggregations to group the data. Preview the data to ensure there is one record per day.

In [9]:
weather_df["datetime"] = weather_df["datetime"].dt.normalize()

# group by datetime
aggregations = {"precipTime": "max", "temperature": "mean", "precipDepth": "max"}
weather_df_grouped = weather_df.groupby("datetime").agg(aggregations)
weather_df_grouped.head(10)

,precipTime,temperature,precipDepth
datetime,,,
2016-01-01,1.0,5.197345,0.0
2016-01-02,1.0,2.567857,0.0
2016-01-03,1.0,3.846429,0.0
2016-01-04,1.0,0.123894,0.0
2016-01-05,6.0,-7.206250,0.0
2016-01-06,6.0,-0.896396,0.0
2016-01-07,6.0,3.180645,0.0
2016-01-08,1.0,4.384091,0.0
2016-01-09,6.0,6.710274,3.0


Note: The examples in this tutorial merge data using Pandas functions and custom aggregations, but the Open Datasets SDK has classes designed to easily merge and enrich data sets. See the [notebook](https://github.com/Azure/OpenDatasetsNotebooks/blob/master/tutorials/data-join/04-nyc-taxi-join-weather-in-pandas.ipynb) for code examples of these design patterns.

### Cleanse data

Merge the existing taxi and holiday data with the new weather data. This time `datetime` is the only key, and again perform a left-join of the data. Run the `describe()` function on the new dataframe to see summary statistics for each field.

In [10]:
taxi_holidays_weather_df = pd.merge(taxi_holidays_df, weather_df_grouped, how="left", on=["datetime"])
taxi_holidays_weather_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,hr_sin,hr_cos,dy_sin,dy_cos,precipTime,temperature,precipDepth
count,24000.000000,24000.000000,24000.000000,12000.000000,12000.000000,12000.000000,12000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,2.400000e+04,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000
mean,1.793625,1.359458,2.798265,-73.746045,40.641902,-73.812426,40.677156,14.603195,6.500000,15.072875,3.236458,13.611000,-0.246484,-2.038304e-02,-0.085070,-0.050450,13.408667,13.876231,1075.977667
std,0.404711,1.033421,2.976438,3.753491,2.069237,3.016449,1.663137,11.596075,3.452124,8.475006,1.964295,6.682823,0.665381,7.043703e-01,0.713593,0.693574,10.330720,9.462154,2849.048787
min,1.000000,0.000000,0.000000,-74.163818,0.000000,-75.167496,0.000000,-83.900000,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000e+00,-0.974928,-0.900969,1.000000,-13.379464,0.000000
25%,2.000000,1.000000,1.020000,-73.961123,40.694324,-73.968376,40.695145,7.880000,3.750000,8.000000,2.000000,9.000000,-0.866025,-7.071068e-01,-0.781831,-0.900969,6.000000,6.591071,0.000000
50%,2.000000,1.000000,1.830000,-73.946201,40.746000,-73.945480,40.746264,11.300000,6.500000,15.000000,3.000000,15.000000,-0.500000,-1.836970e-16,0.000000,-0.222521,6.000000,13.125893,10.000000
75%,2.000000,1.000000,3.430000,-73.918732,40.801911,-73.912468,40.789734,17.300000,9.250000,22.000000,5.000000,19.000000,0.258819,7.071068e-01,0.781831,0.623490,24.000000,22.944737,132.000000
max,2.000000,8.000000,86.700000,0.000000,41.015667,0.000000,41.085476,495.000000,12.000000,30.000000,6.000000,23.000000,1.000000,1.000000e+00,0.974928,1.000000,24.000000,31.303665,9999.000000


From the summary statistics, you see that there are several fields that have outliers or values that will reduce model accuracy. First filter the lat/long fields to be within the same bounds you used for filtering weather data. The `tripDistance` field has some bad data, because the minimum value is negative. The `passengerCount` field has bad data as well, with the max value being 210 passengers. Lastly, the `totalAmount` field has negative values, which don't make sense in the context of our model.

Filter out these anomolies using query functions, and then remove the last few columns unnecesary for training.

Note: since a random sample of 2000 was taken for each month of the taxi data, the statistics may vary each time this is ran.

In [11]:
final_df = taxi_holidays_weather_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88 and \
                                           pickupLongitude>=-74.09 and pickupLongitude<=-73.72 and \
                                           tripDistance>0 and tripDistance<75 and \
                                           passengerCount>0 and passengerCount<100 and \
                                           totalAmount>0")

Call `describe()` again on the data to ensure cleansing worked as expected. The final data is prepared and cleansed, consisting of taxi, holiday, and weather data, and is ready to use for machine learning model training.

In [12]:
final_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,hr_sin,hr_cos,dy_sin,dy_cos,precipTime,temperature,precipDepth
count,11720.000000,11720.000000,11720.000000,11720.000000,11720.000000,11720.000000,11720.000000,11720.000000,11720.000000,11720.000000,11720.000000,11720.000000,11720.000000,1.172000e+04,11720.000000,11720.000000,11720.000000,11720.000000,11720.000000
mean,1.794710,1.354778,2.829323,-73.937425,40.746545,-73.879205,40.713237,14.597789,3.501024,14.890444,3.252645,13.621672,-0.244164,-1.142466e-02,-0.090539,-0.049453,12.066980,10.267549,190.603498
std,0.403931,1.025023,2.899549,0.041218,0.056494,2.048760,1.130176,10.594542,1.707714,8.454712,1.967197,6.721303,0.666575,7.042813e-01,0.713570,0.693007,10.146518,8.484011,1215.018267
min,1.000000,1.000000,0.010000,-74.074181,40.573597,-74.186638,0.000000,0.010000,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000e+00,-0.974928,-0.900969,1.000000,-13.379464,0.000000
25%,2.000000,1.000000,1.060000,-73.961384,40.694648,-73.968775,40.695228,8.160000,2.000000,8.000000,2.000000,9.000000,-0.866025,-7.071068e-01,-0.781831,-0.900969,1.000000,3.504580,0.000000
50%,2.000000,1.000000,1.880000,-73.946762,40.745876,-73.946011,40.746073,11.300000,4.000000,15.000000,4.000000,15.000000,-0.500000,-1.836970e-16,-0.433884,-0.222521,6.000000,10.130357,3.000000
75%,2.000000,1.000000,3.490000,-73.919127,40.801327,-73.912937,40.788689,17.300000,5.000000,22.000000,5.000000,19.000000,0.258819,7.071068e-01,0.781831,0.623490,24.000000,17.239744,41.000000
max,2.000000,6.000000,52.800000,-73.744164,40.879837,0.000000,41.025719,223.890000,6.000000,30.000000,6.000000,23.000000,1.000000,1.000000e+00,0.974928,1.000000,24.000000,26.524107,9999.000000


## Train a model

The data is ready to train a machine learning model.

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

### Training Function

Define a function that can be used to create a model pipeline that can be trained and then used for scoring. This pipeline has 2 steps: preprocessing and model training.

<b>Preprocessing Stages:</b>
The preprocessing step of the pipeline also has 2 stages, one for numerical features and one for categorical features.
For the numerical features, let's fill in any blanks with 0's. While the training data may not have any nulls in the these fields, future data that is scored may and this step will take care of those for us. Optionally, a scaler transformation could be added in this step as well. Similarly for the categorical variables, let's have the null values filled with "MISSING". Additionally to the categorical variables, these will need to be one hot encoded, so we will include that step in our pipeline.

<b>Model Training Stage:</b>
An input parameter will determine which type of model of train. Let's test out a linear regression and random forest model to start. 

The two steps are put together into the pipeline which is what the function is returning.

In [14]:
def createClassModel(algo_name, catg, nums):
  numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value=0))])
  
  categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value="MISSING")), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
  
  preprocesser = ColumnTransformer(transformers=[('num', numeric_transformer, nums), ('cat', categorical_transformer, catg)])
  
  if algo_name == 'linear_regression':
    model=Ridge(alpha=100)
  elif algo_name == 'random_forest':
    model = RandomForestRegressor()
  else:
    pass
  ModelPipeline = Pipeline(steps=[('preprocessor', preprocesser), ("model", model)])
  return ModelPipeline

Let's define the arguments that will be passed to the function. `catg_cols` is a list of the categorical variables that will be transformed in our processing step. `num_cols` is a list of the numerical variables that will be transformed in our processing step. Let's define the target column as `label` so it can be used in future steps as well.

In [15]:
catg_cols = ["vendorID", "month_num", "day_of_month", "normalizeHolidayName", "isPaidTimeOff"]
num_cols = ["passengerCount", "tripDistance", "precipTime", "temperature", "precipDepth", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]
label = ["totalAmount"]

The training is ready to begin, but first, let's make sure that the categorical variables are strings in our dataframe to ensure no errors in our pipeline. 

Next, the data is split into training and test sets by using the `train_test_split()` function in the `scikit-learn` library. The `test_size` parameter determines the percentage of data to allocate to testing. The `random_state` parameter sets a seed to the random number generator, so that your train-test splits are deterministic.

The training will happen in the for loop so that both algorithms can be tested. The createClassModel funtion is called to retreive the pipeline that can then be trained using the training dataset. 

Once trained, the test dataset is then ran through the model to test the model's performance. Using various functions from sklearn.metrics, the R2 score, MAPE, and RMSE can be used to measure model performance.

In [16]:
# make sure categorical columns are strings
final_df[catg_cols] = final_df[catg_cols].astype("str")

# split data
X_train, X_test, y_train, y_test = train_test_split(final_df.drop(label, axis=1), final_df[label], test_size=0.2, random_state=222)

# test 2 algorithms
for algorithmname in ["linear_regression", 'random_forest']:
    fitPipeline = createClassModel(algorithmname, catg_cols, num_cols) # get pipeline
    fitPipeline.fit(X_train, y_train.values.ravel())                   # fit pipeine

    y_pred = fitPipeline.predict(X_test)                               # score with fitted pipeline

    # Evaluate
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(algorithmname)
    print("R2:", r2)
    print("MAPE:", mape)
    print("RMSE:", rmse)
    print()

linear_regression
R2: 0.8939180427845623
MAPE: 0.15217635144070302
RMSE: 3.409148681526453

random_forest
R2: 0.8540936112427824
MAPE: 0.15527304667688627
RMSE: 3.998179929258663

